# Librerias

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
os.getcwd()

'c:\\Users\\sdsal\\OneDrive - Universidad de los Andes\\2023-2\\Tesis\\codigo'

In [4]:
os.chdir(path="../datos/unificado/")

# Funciones Auxiliares

In [5]:
def obtain_unique_locations(list):
    df = pd.DataFrame({'NOMBRE_BARRIO':[],'LOCALIDAD':[]})
    
    for i in list:
        current_df = pd.read_csv(i)
        temporal_df = pd.DataFrame()
        
        # Extraer la parte del barrio sin la "E-XX"
        temporal_df['NOMBRE_BARRIO'] = current_df['BARRIO'].str.extract(r'(.*) E-\d+')
        # Extraer el número después de "E-"
        temporal_df['LOCALIDAD'] = current_df['BARRIO'].str.extract(r'E-(\d+)')
        #temporal_df.head(1)
        df = pd.concat([df,temporal_df],axis=0)
    
    df = df.drop_duplicates(subset=['NOMBRE_BARRIO','LOCALIDAD'])
    return df

In [6]:
def obtain_not_reported_neighbors(path,reports):
    df_list = []
    
    for i in range(1,20):
    
        current_location = reports[reports['LOCALIDAD']==f'{i}']
        reported_neighbors = current_location['NOMBRE_BARRIO']
        
        df_upzs_info = pd.read_excel(path,f'{i}')
        known_neighbors = df_upzs_info['BARRIO'].tolist()
        
        list_neighbors = [j for j in reported_neighbors if j not in known_neighbors]
        list_locations = [i] * len(list_neighbors)     
                
        temporal_df =pd.DataFrame({'NOMBRE_BARRIO':list_neighbors,'LOCALIDAD':list_locations})    
        df_list.append(temporal_df)
        
    df = pd.concat(df_list, axis=0, ignore_index=True)
    return df

# Desarrollo

Lista de delitos a analizar

In [7]:
lista_delitos = ['homicidios.csv',
                 'hurto_a_personas.csv',
                 'hurto_de_automotores.csv',
                 'hurto_de_celulares.csv',
                 'hurto_de_motos.csv']

El DataFrame resultante contiene todos los nombres de los barrios cuyos reportes existen.

In [8]:
unique_locations = obtain_unique_locations(lista_delitos)

C:\Users\sdsal\AppData\Local\Temp\ipykernel_20156\1820550837.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv(i)


In [9]:
unique_locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4326 entries, 0 to 40271
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   NOMBRE_BARRIO  4325 non-null   object
 1   LOCALIDAD      4325 non-null   object
dtypes: object(2)
memory usage: 101.4+ KB


Se definen los barrios que componen a cada UPZ.

In [10]:
df_not_upz = obtain_not_reported_neighbors('listado_barrios_por_upz.xlsx',unique_locations)
df_not_upz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997 entries, 0 to 2996
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   NOMBRE_BARRIO  2997 non-null   object
 1   LOCALIDAD      2997 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [11]:
nombre_archivo_excel = 'barrios_no_encontrados.xlsx'

df_not_upz.to_excel(nombre_archivo_excel, index=False)